In [1]:
# imports
import numpy as np
import pandas as pd
from fastai.tabular.all import *
# from scipy.spatial.transform import Rotation as R

In [2]:
# helpers
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)

def split(df, ratio, length): 
    marker = int(length * ratio)
    train_idx = np.where(df[:marker])[0]
    valid_idx = np.where(df[marker:])[0]
    splits = (list(train_idx),list(valid_idx)); return splits

FINGERS = ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']
BONES = ['Metacarpal', 'Proximal', 'Intermediate', 'Distal']
AXES = ['X', 'Y', 'Z']

In [3]:
# columns
input_cols = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6']
output_cols = [f+'_'+b+'_'+axis for f in FINGERS for b in BONES for axis in AXES]

In [4]:
# data
num = input("Dataset number to use") # implement plural?
dirc = pd.read_csv('./data/'+num+'_dirc.csv')
dirc = dirc.loc[(dirc != 0).any(1)]
semg = pd.read_csv('./data/'+num+'_semg.csv') 
semg = semg.loc[(semg != 0).any(1)] 

# Train

In [5]:
df = pd.concat([semg, dirc], axis=1)

In [6]:
df

,E1,E2,E3,E4,E5,E6,Thumb_Metacarpal_X,Thumb_Metacarpal_Y,Thumb_Metacarpal_Z,Thumb_Proximal_X,...,Pinky_Metacarpal_Z,Pinky_Proximal_X,Pinky_Proximal_Y,Pinky_Proximal_Z,Pinky_Intermediate_X,Pinky_Intermediate_Y,Pinky_Intermediate_Z,Pinky_Distal_X,Pinky_Distal_Y,Pinky_Distal_Z
57,4017.0,4010.0,4021.0,4095.0,1291.0,3605.0,0.0,0.0,0.0,-0.975,...,-0.400,-0.380,0.766,-0.519,-0.559,0.773,-0.302,-0.720,0.693,-0.028
58,4017.0,4011.0,4021.0,4095.0,1291.0,3605.0,0.0,0.0,0.0,-0.943,...,-0.346,-0.463,0.877,-0.126,-0.514,0.855,-0.069,-0.709,0.691,0.141
59,4017.0,4011.0,4018.0,4095.0,1291.0,3605.0,0.0,0.0,0.0,-0.870,...,-0.303,-0.437,0.886,-0.159,-0.469,0.882,-0.049,-0.530,0.803,0.274
60,4010.0,3999.0,4023.0,4095.0,1285.0,2807.0,0.0,0.0,0.0,-0.888,...,-0.227,-0.417,0.882,-0.219,-0.429,0.890,-0.155,-0.460,0.827,0.322
61,4010.0,4016.0,4023.0,4095.0,1285.0,2807.0,0.0,0.0,0.0,-0.973,...,-0.221,-0.394,0.881,-0.263,-0.394,0.881,-0.263,-0.413,0.796,0.441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,1415.0,134.0,145.0,4095.0,1319.0,369.0,0.0,0.0,0.0,-0.653,...,-0.907,-0.541,-0.840,0.054,-0.268,0.037,0.963,0.236,0.703,0.671
1927,1625.0,128.0,145.0,4095.0,1319.0,369.0,0.0,0.0,0.0,-0.643,...,-0.895,-0.567,-0.822,0.054,-0.284,0.040,0.958,0.239,0.691,0.683
1928,1625.0,128.0,133.0,4095.0,1318.0,464.0,0.0,0.0,0.0,-0.614,...,-0.898,-0.607,-0.794,0.010,-0.300,-0.057,0.952,0.133,0.483,0.865
1929,1985.0,153.0,133.0,4095.0,1318.0,464.0,0.0,0.0,0.0,-0.574,...,-0.919,-0.605,-0.796,-0.015,-0.303,-0.106,0.947,0.091,0.401,0.912


In [7]:
split_ratio = 0.7 # number is train
to = TabularPandas(df, procs=None, cat_names=None, cont_names=input_cols, y_names=output_cols, splits=split(df, split_ratio, df.shape[0]))
dls = to.dataloaders(1024)

In [8]:
y = to.train.y
y.min(), y.max()

(0.0, 0.0)

In [9]:
learn = tabular_learner(dls, y_range=(-1, 1), n_out=60, loss_func=F.mse_loss)

In [10]:
learn.lr_find()

IndexError: index 0 is out of bounds for dimension 0 with size 0

In [ ]:
learn.fit_one_cycle(n_epoch=4, lr_max=1e-3)

epoch,train_loss,valid_loss,time
0,0.325962,0.295022,00:01
1,0.260722,0.197618,00:00
2,0.201621,0.164695,00:00
3,0.173947,0.160024,00:00


In [ ]:
preds,targs = learn.get_preds()

In [ ]:
r_mse(preds,targs)

0.40003

# Export

In [ ]:
learn.export()

# Inference

In [ ]:
pdf = pd.DataFrame([[0, 0, 356.0, 459.0, 536.0, 431.0]], columns=input_cols)
row, clas, probs = learn.predict(pdf.iloc[0]) # df 넣어야 하네. input만 들어있어도 될까, 다 들어있어야 할까?

In [ ]:
row.show()

,E1,E2,E3,E4,E5,E6,Thumb_Metacarpal_X,Thumb_Metacarpal_Y,Thumb_Metacarpal_Z,Thumb_Proximal_X,Thumb_Proximal_Y,Thumb_Proximal_Z,Thumb_Intermediate_X,Thumb_Intermediate_Y,Thumb_Intermediate_Z,Thumb_Distal_X,Thumb_Distal_Y,Thumb_Distal_Z,Index_Metacarpal_X,Index_Metacarpal_Y,Index_Metacarpal_Z,Index_Proximal_X,Index_Proximal_Y,Index_Proximal_Z,Index_Intermediate_X,Index_Intermediate_Y,Index_Intermediate_Z,Index_Distal_X,Index_Distal_Y,Index_Distal_Z,Middle_Metacarpal_X,Middle_Metacarpal_Y,Middle_Metacarpal_Z,Middle_Proximal_X,Middle_Proximal_Y,Middle_Proximal_Z,Middle_Intermediate_X,Middle_Intermediate_Y,Middle_Intermediate_Z,Middle_Distal_X,Middle_Distal_Y,Middle_Distal_Z,Ring_Metacarpal_X,Ring_Metacarpal_Y,Ring_Metacarpal_Z,Ring_Proximal_X,Ring_Proximal_Y,Ring_Proximal_Z,Ring_Intermediate_X,Ring_Intermediate_Y,Ring_Intermediate_Z,Ring_Distal_X,Ring_Distal_Y,Ring_Distal_Z,Pinky_Metacarpal_X,Pinky_Metacarpal_Y,Pinky_Metacarpal_Z,Pinky_Proximal_X,Pinky_Proximal_Y,Pinky_Proximal_Z,Pinky_Intermediate_X,Pinky_Intermediate_Y,Pinky_Intermediate_Z,Pinky_Distal_X,Pinky_Distal_Y,Pinky_Distal_Z
0,0.0,0.0,356.0,459.0,536.0,431.0,0.006468,0.02652,0.001305,-0.553461,-0.118103,-0.420492,-0.435771,-0.109493,-0.437243,-0.261515,-0.269954,-0.50578,-0.597962,-0.040552,-0.367703,-0.522396,0.006851,-0.502267,-0.364578,-0.049652,-0.33908,-0.394821,-0.232847,-0.243695,-0.519056,-0.050474,-0.627423,-0.270577,-0.12013,-0.649609,-0.24256,-0.132888,-0.268017,-0.167898,-0.200596,-0.200169,-0.470587,-0.067264,-0.66141,-0.279898,-0.298513,-0.532372,-0.08385,-0.268477,-0.027445,0.017115,-0.272119,0.147171,-0.218245,-0.07315,-0.616949,-0.127595,-0.276959,-0.435903,-0.178336,-0.315801,-0.130769,-0.094681,-0.350055,0.194433


In [ ]:
# clas meaningless since everything's continuous
probs

tensor([ 0.0065,  0.0265,  0.0013, -0.5535, -0.1181, -0.4205, -0.4358, -0.1095,
        -0.4372, -0.2615, -0.2700, -0.5058, -0.5980, -0.0406, -0.3677, -0.5224,
         0.0069, -0.5023, -0.3646, -0.0497, -0.3391, -0.3948, -0.2328, -0.2437,
        -0.5191, -0.0505, -0.6274, -0.2706, -0.1201, -0.6496, -0.2426, -0.1329,
        -0.2680, -0.1679, -0.2006, -0.2002, -0.4706, -0.0673, -0.6614, -0.2799,
        -0.2985, -0.5324, -0.0838, -0.2685, -0.0274,  0.0171, -0.2721,  0.1472,
        -0.2182, -0.0732, -0.6169, -0.1276, -0.2770, -0.4359, -0.1783, -0.3158,
        -0.1308, -0.0947, -0.3501,  0.1944])

In [ ]:
a, b = learn.get_preds()
r_mse(a,b)
